# Renderer

Excel file generator for the optimized schedule.

In [60]:
import numpy as np

day_to_int = { "Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6 }

# list of classes
classes=['Advanced Solid State Physics','Photonics Review','Multiscale Methods','Neuroscience','Spacecraft and Mission Design',
    'Robotics','Intro to Solid State Physics','Optimization Methods','Math. Modeling in Biology',
    'Comp.Chemistry and Materials Modeling','Gene Engineering','Electric Machines', 'Data Security', 
    'Intro to Num. Simulations', 'Ideas to Impact', 'Petrophysics and Reservoir Engineering', 
    'Structure and Properties of Materials', 'Machine Learning', 'Energy Colloquim', 'Skoltech Colloquium',
    'Functional Genomics']

# schedule format
schedule = {
    "Monday": 
    [['Advanced Solid State Physics', 'Optimization Methods', 'Math. Modeling in Biology'],
     ['Advanced Solid State Physics', 'Intro to Solid State Physics', 'Optimization Methods', 'Machine Learning'],
     ['Multiscale Methods', 'Robotics', 'Comp.Chemistry and Materials Modeling', 'Intro to Num. Simulations']],
    
    "Tuesday":
    [['Photonics Review', 'Optimization Methods', 'Structure and Properties of Materials', 'Machine Learning', 'Energy Colloquim'],
     ['Gene Engineering', 'Electric Machines', 'Intro to Num. Simulations', 'Petrophysics and Reservoir Engineering'],
     ['Robotics', 'Comp.Chemistry and Materials Modeling', 'Gene Engineering', 'Data Security', 'Ideas to Impact']],
    
    "Thursday":
    [['Photonics Review', 'Intro to Solid State Physics', 'Data Security'],
     ['Neuroscience', 'Optimization Methods', 'Petrophysics and Reservoir Engineering', 'Skoltech Colloquium'],
     ['Advanced Solid State Physics', 'Intro to Solid State Physics', 'Math. Modeling in Biology', 'Comp.Chemistry and Materials Modeling', 'Ideas to Impact']],

    "Friday":
    [['Photonics Review', 'Multiscale Methods', 'Neuroscience', 'Ideas to Impact', 'Functional Genomics'],
     ['Robotics', 'Electric Machines', 'Intro to Num. Simulations', 'Petrophysics and Reservoir Engineering', 'Structure and Properties of Materials'],
     ['Multiscale Methods', 'Math. Modeling in Biology', 'Electric Machines']]
}

# rooms list
# without '450 Space Lab' because we should only assign space to it
rooms = ['402', '402', '403', '407', '408', '421', '423', '404CL', '404 CL']

# dates for stub
dates = ['Feb 1', 'Feb 2', 'Feb 3', 'Feb 4', 'Feb 5']


In [75]:
import xlsxwriter
from datetime import datetime, timedelta

workbook = xlsxwriter.Workbook('schedule.xlsx')
ws = workbook.add_worksheet()

glob = {
    'bold': 1,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Arial',
    'font_size': 10,
    'fg_color': 'white'}

global_format = workbook.add_format(glob)
frm_light = workbook.add_format(glob)
frm_dark = workbook.add_format(glob)
frm_light.set_pattern(1)
frm_dark.set_pattern(1)
frm_light.set_bg_color('#FFFFCC')
frm_dark.set_bg_color('#FFFF99')

class_format = workbook.add_format(glob)
class_format.set_rotation(90)
class_format.set_pattern(1)
class_format.set_bg_color('#FFCCFF')
class_format.set_text_wrap()

for i in range(26):
    for j in range(32):
        ws.write(i, j, None, global_format)

ws.set_row(0, 30)
ws.set_row(1, 30)
ws.set_row(2, 30)
ws.set_row(3, 30)

ws.merge_range('A1:A4', None)

def add_time_col(ws):
    ws.set_column('A:A', 10)
    date_time = datetime.strptime('2016-02-01 09:00:00.000',
                              '%Y-%m-%d %H:%M:%S.%f')
    dt_str = date_time.strftime("%H:%M")    
    
    # 22 time intervals
    for i in xrange(4, 4 + 22):
        ws.set_row(i, 17)
        if (i - 3) == 8:
            next_dt = date_time + timedelta(0,12*60*60+30*60)
        else:
            next_dt = date_time + timedelta(0,30*60)
            
        value = dt_str + ' - ' + next_dt.strftime("%H:%M")
        date_time = next_dt
        dt_str = date_time.strftime("%H:%M")
        
        if ((i - 3) % 7) == 0 and (i-3) < 15:
            ws.write_string(i, 0, value, frm_dark)
        else:
            ws.write_string(i, 0, value, frm_light)
            
add_time_col(ws)
    
#def init_classes():
    
def add_day(row_num, col_num, day_name, date_str):
    width = 0
    
    for i in xrange(3):
        width = max(len(schedule[day_name][i]), width)
    
    width = max(6, width)
    
    ws.merge_range(row_num, col_num, row_num, col_num + width - 1, date_str, frm_light)
    row_num += 1
    ws.merge_range(row_num, col_num, row_num, col_num + width - 1, day_name, global_format)
    row_num += 1
    
    for i in xrange(width):
        ws.write_string(row_num, col_num + i, rooms[i], frm_dark)
    
    row_num += 1
    
    for i in xrange(3):
        
        for j, c in enumerate(schedule[day_name][i]):
            ws.merge_range(row_num, col_num + j, row_num + 5, col_num+j, c, class_format)
        
        it = len(schedule[day_name][i])
        
        for j in xrange(width - it):
            ws.merge_range(row_num, col_num + j + it, row_num + 5, col_num + j + it, None)
            
        row_num += 7
    
add_day(1, 1, "Monday", dates[0])

workbook.close()